# Helper Function Discovery
> Functions to discover and extract helper functions:

In [ ]:
#| default_exp cli.helper_discovery

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from dataclasses import dataclass
import inspect
from typing import Dict, List, Tuple, Any, Optional
from cjm_fasthtml_tailwind.cli.example_discovery import ExampleInfo
from cjm_fasthtml_tailwind.cli.utils import (
    discover_utility_modules, extract_helper_names_from_test
) 

In [ ]:
#| export
@dataclass
class HelperInfo:
    """Information about a discovered helper function."""
    name: str  # Function name (e.g., 'pad', 'margin', 'combine_classes')
    module_name: str  # Module where it was found
    function: Any  # The actual function object
    signature: str  # Function signature
    docstring: str  # Function docstring
    source: str  # Source code of the function

In [ ]:
#| export
def get_helper_examples(
    module_name: str  # Name of the module to inspect
) -> Optional[ExampleInfo]:  # ExampleInfo object or None if not found
    """Get the helper examples test function for a module."""
    # Find the module
    for name, module in discover_utility_modules():
        if name == module_name:
            # Look for test_<module>_helper_examples function
            func_name = f'test_{module_name}_helper_examples'
            
            if hasattr(module, func_name):
                try:
                    func = getattr(module, func_name)
                    if callable(func):
                        source = inspect.getsource(func) if hasattr(inspect, 'getsource') else ""
                        docstring = inspect.getdoc(func) or "No documentation available"
                        
                        return ExampleInfo(
                            name=func_name,
                            module_name=module_name,
                            feature='helper',
                            function=func,
                            source=source,
                            docstring=docstring
                        )
                except (AttributeError, OSError):
                    pass
    
    return None

In [ ]:
#| export
def get_module_helpers(
    module_name: str  # Name of the module to inspect
) -> List[HelperInfo]:  # List of HelperInfo objects
    """Get helper functions from a module based on its test_<module>_helper_examples function."""
    helpers = []
    
    # First, get the helper examples test function
    helper_example = get_helper_examples(module_name)
    if not helper_example:
        return helpers
    
    # Extract helper function names from the test
    helper_names = extract_helper_names_from_test(helper_example.source)
    
    # Now get the actual module and extract info for each helper
    for name, module in discover_utility_modules():
        if name == module_name:
            for helper_name in helper_names:
                if hasattr(module, helper_name):
                    try:
                        func = getattr(module, helper_name)
                        if callable(func):
                            signature = str(inspect.signature(func)) if hasattr(inspect, 'signature') else ""
                            docstring = inspect.getdoc(func) or "No documentation available"
                            source = inspect.getsource(func) if hasattr(inspect, 'getsource') else ""
                            
                            helpers.append(HelperInfo(
                                name=helper_name,
                                module_name=module_name,
                                function=func,
                                signature=signature,
                                docstring=docstring,
                                source=source
                            ))
                    except (AttributeError, OSError, TypeError):
                        pass
            break
    
    return sorted(helpers, key=lambda x: x.name)

## Export

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()